# Análise de Quebras Estruturais com Seções de Lévy

Este notebook demonstra como usar o método das seções de Lévy para:
1. Detectar mudanças de regime de volatilidade em séries temporais
2. Extrair features robustas para machine learning
3. Comparar com o desempenho do TSFresh

Baseado no artigo de Figueiredo et al. (2022) e na implementação para o projeto CrunchDAO SB.

In [ ]:
# Imports necessários
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Importar nossa implementação
from src.features.levy_sections import LevySectionsAnalyzer, analyze_structural_breaks_for_series

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Carregamento e Preparação dos Dados

In [ ]:
# Função auxiliar para carregar dados
def load_data(ticker, start_date, end_date):
    """Carrega dados do Yahoo Finance e calcula retornos"""
    print(f"Carregando {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data['Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    return data.dropna()

# Carregar múltiplos ativos para comparação
tickers = {
    '^BVSP': 'IBOVESPA',
    '^GSPC': 'S&P 500',
    'BTC-USD': 'Bitcoin',
    'PETR4.SA': 'Petrobras'
}

start_date = '2018-01-01'
end_date = '2024-12-31'

data_dict = {}
for ticker, name in tickers.items():
    try:
        data_dict[ticker] = load_data(ticker, start_date, end_date)
        print(f"✓ {name}: {len(data_dict[ticker])} observações")
    except:
        print(f"✗ Erro ao carregar {name}")

## 2. Análise Exploratória dos Retornos

In [ ]:
# Visualizar retornos e volatilidade
fig, axes = plt.subplots(len(data_dict), 2, figsize=(15, 3*len(data_dict)))

for i, (ticker, data) in enumerate(data_dict.items()):
    # Retornos
    ax1 = axes[i, 0] if len(data_dict) > 1 else axes[0]
    ax1.plot(data.index, data['Returns'], linewidth=0.5)
    ax1.set_title(f'{tickers[ticker]} - Retornos Diários')
    ax1.set_ylabel('Log-retorno')
    
    # Volatilidade realizada (janela de 20 dias)
    ax2 = axes[i, 1] if len(data_dict) > 1 else axes[1]
    vol = data['Returns'].rolling(20).std() * np.sqrt(252)
    ax2.plot(data.index, vol)
    ax2.set_title(f'{tickers[ticker]} - Volatilidade Anualizada (20d)')
    ax2.set_ylabel('Volatilidade')

plt.tight_layout()
plt.show()

# Estatísticas descritivas
stats_df = pd.DataFrame()
for ticker, data in data_dict.items():
    stats = {
        'Média': data['Returns'].mean() * 252,
        'Volatilidade': data['Returns'].std() * np.sqrt(252),
        'Sharpe': (data['Returns'].mean() * 252) / (data['Returns'].std() * np.sqrt(252)),
        'Curtose': data['Returns'].kurtosis(),
        'Assimetria': data['Returns'].skew(),
        'VaR 5%': data['Returns'].quantile(0.05),
        'CVaR 5%': data['Returns'][data['Returns'] <= data['Returns'].quantile(0.05)].mean()
    }
    stats_df[tickers[ticker]] = stats

print("\nEstatísticas dos Retornos:")
print(stats_df.round(4).T)

## 3. Aplicação das Seções de Lévy

In [ ]:
# Escolher ativo para análise detalhada
selected_ticker = '^BVSP'
data = data_dict[selected_ticker]
returns = data['Returns'].values

print(f"Analisando {tickers[selected_ticker]}...")
print(f"Período: {data.index[0].strftime('%Y-%m-%d')} a {data.index[-1].strftime('%Y-%m-%d')}")
print(f"Total de observações: {len(returns)}")

In [ ]:
# Testar diferentes valores de tau
tau_values = [0.001, 0.005, 0.01, 0.02, 0.05]
results_by_tau = {}

for tau in tau_values:
    analyzer = LevySectionsAnalyzer(tau=tau, q=5)
    levy_result = analyzer.compute_levy_sections(returns)
    
    results_by_tau[tau] = {
        'analyzer': analyzer,
        'result': levy_result,
        'n_sections': len(levy_result.S_tau),
        'mean_duration': np.mean(levy_result.durations),
        'std_duration': np.std(levy_result.durations),
        'cv_duration': np.std(levy_result.durations) / np.mean(levy_result.durations)
    }

# Resumo dos resultados
summary_df = pd.DataFrame(results_by_tau).T
summary_df = summary_df[['n_sections', 'mean_duration', 'std_duration', 'cv_duration']]
print("\nResumo para diferentes valores de tau:")
print(summary_df.round(2))

## 4. Detecção de Quebras Estruturais

In [ ]:
# Análise detalhada com tau = 0.005
tau_selected = 0.005
analyzer = results_by_tau[tau_selected]['analyzer']
levy_result = results_by_tau[tau_selected]['result']

# Detectar quebras
breaks_info = analyzer.detect_structural_breaks()

print(f"\nAnálise com tau = {tau_selected}:")
print(f"Quebras estruturais detectadas: {len(breaks_info['breaks'])}")

# Mapear quebras para datas
for i, brk in enumerate(breaks_info['breaks']):
    section_idx = brk['index']
    if section_idx < len(levy_result.start_indices):
        date_idx = levy_result.start_indices[section_idx]
        if date_idx < len(data.index):
            break_date = data.index[date_idx]
            print(f"\nQuebra {i+1}:")
            print(f"  Data: {break_date.strftime('%Y-%m-%d')}")
            print(f"  Mudança na duração média: {brk['mean_before']:.1f} → {brk['mean_after']:.1f} dias")
            print(f"  Razão de mudança: {brk['change_ratio']:.3f}")

In [ ]:
# Visualização das quebras
fig = analyzer.plot_analysis(breaks_info, title_prefix=tickers[selected_ticker])
plt.show()

## 5. Comparação com Eventos de Mercado Conhecidos

In [ ]:
# Eventos importantes no período
market_events = [
    ('2020-02-20', '2020-03-23', 'COVID-19 Crash', 'red'),
    ('2021-01-27', '2021-02-05', 'GameStop/Reddit Rally', 'orange'),
    ('2022-02-24', '2022-03-08', 'Invasão da Ucrânia', 'darkred'),
    ('2023-03-10', '2023-03-20', 'Crise SVB/Credit Suisse', 'purple')
]

# Plotar série temporal com eventos e quebras detectadas
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), height_ratios=[2, 1])

# Subplot 1: Preço e eventos
ax1.plot(data.index, data['Close'], label='Preço de Fechamento', color='black', linewidth=1)

# Adicionar eventos conhecidos
for start, end, event, color in market_events:
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    if start_date >= data.index[0] and end_date <= data.index[-1]:
        ax1.axvspan(start_date, end_date, alpha=0.3, color=color, label=event)

# Adicionar quebras detectadas
for brk in breaks_info['breaks']:
    section_idx = brk['index']
    if section_idx < len(levy_result.start_indices):
        date_idx = levy_result.start_indices[section_idx]
        if date_idx < len(data.index):
            break_date = data.index[date_idx]
            ax1.axvline(break_date, color='green', linestyle='--', linewidth=2, alpha=0.7)

ax1.set_title(f'{tickers[selected_ticker]} - Eventos de Mercado vs. Quebras Detectadas (Linhas Verdes)')
ax1.set_ylabel('Preço')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Subplot 2: Durações das seções
section_dates = [data.index[idx] if idx < len(data.index) else data.index[-1] 
                 for idx in levy_result.start_indices]
ax2.plot(section_dates[:len(levy_result.durations)], levy_result.durations, 
         'o-', markersize=4, label='Duração das Seções')
ax2.set_ylabel('Duração (dias)')
ax2.set_xlabel('Data')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Extração de Features para Machine Learning

In [ ]:
# Extrair features de todos os ativos
features_all = {}

for ticker in data_dict.keys():
    returns = data_dict[ticker]['Returns'].values
    
    # Usar tau = 0.005 como padrão
    analyzer = LevySectionsAnalyzer(tau=0.005, q=5)
    analyzer.compute_levy_sections(returns)
    features = analyzer.extract_features()
    
    features_all[tickers[ticker]] = features

# Criar DataFrame com features
features_df = pd.DataFrame(features_all).T

# Selecionar features mais importantes
important_features = [
    'levy_duration_mean',
    'levy_duration_cv',
    'levy_duration_kurtosis',
    'levy_norm_kurtosis',
    'levy_shapiro_pvalue',
    'levy_duration_autocorr'
]

print("\nFeatures de Lévy para cada ativo:")
print(features_df[important_features].round(4))

In [ ]:
# Visualizar correlação entre features
plt.figure(figsize=(10, 8))
corr_matrix = features_df[important_features].T.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlação entre Ativos (Features de Lévy)')
plt.tight_layout()
plt.show()

## 7. Análise de Robustez: Variação de Parâmetros

In [ ]:
# Análise de sensibilidade aos parâmetros
q_values = [3, 5, 10, 15]
tau = 0.005

sensitivity_results = []

for q in q_values:
    analyzer = LevySectionsAnalyzer(tau=tau, q=q)
    levy_result = analyzer.compute_levy_sections(returns)
    breaks_info = analyzer.detect_structural_breaks()
    
    sensitivity_results.append({
        'q': q,
        'n_sections': len(levy_result.S_tau),
        'n_breaks': len(breaks_info['breaks']),
        'mean_duration': np.mean(levy_result.durations),
        'cv_duration': np.std(levy_result.durations) / np.mean(levy_result.durations)
    })

sensitivity_df = pd.DataFrame(sensitivity_results)
print("\nSensibilidade ao parâmetro q:")
print(sensitivity_df)

## 8. Aplicação Prática: Sinal de Trading

In [ ]:
# Simular estratégia baseada em quebras estruturais
# Ideia: Reduzir exposição quando detectada quebra para regime de alta volatilidade

analyzer = LevySectionsAnalyzer(tau=0.005, q=5)
levy_result = analyzer.compute_levy_sections(returns)
breaks_info = analyzer.detect_structural_breaks()

# Criar série de sinais
signals = pd.Series(1, index=data.index)  # 1 = long, 0 = flat

for brk in breaks_info['breaks']:
    if brk['change_ratio'] < 0.7:  # Aumento de volatilidade
        section_idx = brk['index']
        if section_idx < len(levy_result.start_indices):
            date_idx = levy_result.start_indices[section_idx]
            if date_idx < len(data.index):
                break_date = data.index[date_idx]
                # Ficar flat por 20 dias após quebra
                end_date = break_date + timedelta(days=20)
                signals[break_date:end_date] = 0

# Calcular retornos da estratégia
strategy_returns = data['Returns'] * signals.shift(1)  # Shift para evitar look-ahead bias
buy_hold_returns = data['Returns']

# Performance cumulativa
cumulative_strategy = (1 + strategy_returns).cumprod()
cumulative_buyhold = (1 + buy_hold_returns).cumprod()

# Plotar resultados
plt.figure(figsize=(15, 8))
plt.plot(cumulative_buyhold.index, cumulative_buyhold.values, 
         label='Buy & Hold', linewidth=2)
plt.plot(cumulative_strategy.index, cumulative_strategy.values, 
         label='Estratégia Lévy', linewidth=2, alpha=0.8)

# Marcar períodos flat
flat_periods = signals[signals == 0]
for date in flat_periods.index:
    plt.axvline(date, color='red', alpha=0.1, linewidth=1)

plt.title('Comparação: Buy & Hold vs. Estratégia baseada em Quebras de Lévy')
plt.ylabel('Retorno Cumulativo')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Métricas de performance
from scipy import stats as scipy_stats

def calculate_metrics(returns):
    total_return = (1 + returns).prod() - 1
    annual_return = (1 + total_return) ** (252 / len(returns)) - 1
    annual_vol = returns.std() * np.sqrt(252)
    sharpe = annual_return / annual_vol if annual_vol > 0 else 0
    max_dd = (returns.cumsum() - returns.cumsum().cummax()).min()
    
    return {
        'Retorno Total': f"{total_return:.2%}",
        'Retorno Anual': f"{annual_return:.2%}",
        'Volatilidade Anual': f"{annual_vol:.2%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Max Drawdown': f"{max_dd:.2%}"
    }

metrics_bh = calculate_metrics(buy_hold_returns)
metrics_strategy = calculate_metrics(strategy_returns)

metrics_df = pd.DataFrame({
    'Buy & Hold': metrics_bh,
    'Estratégia Lévy': metrics_strategy
})

print("\nMétricas de Performance:")
print(metrics_df)

## 9. Conclusões e Próximos Passos

### Principais Descobertas:

1. **Detecção de Quebras**: O método das seções de Lévy detectou com sucesso várias quebras estruturais que coincidem com eventos de mercado conhecidos.

2. **Gaussianização**: As somas seccionais normalizadas apresentam distribuição mais próxima da normal, validando a teoria.

3. **Features Robustas**: As features extraídas capturam características importantes da dinâmica de volatilidade.

4. **Aplicação Prática**: Uma estratégia simples baseada nas quebras detectadas pode melhorar o perfil de risco-retorno.

### Próximos Passos:

1. **Integração com TSFresh**: Combinar features de Lévy com features do TSFresh para melhorar modelos de classificação.

2. **Otimização de Parâmetros**: Desenvolver método sistemático para escolha ótima de τ e q.

3. **Backtesting Robusto**: Testar estratégias em múltiplos ativos e períodos.

4. **Machine Learning**: Usar as quebras detectadas como labels para treinar modelos preditivos.

In [ ]:
# Salvar resultados principais
output_data = {
    'features': features_df,
    'breaks_summary': pd.DataFrame(breaks_info['breaks']),
    'performance_metrics': metrics_df
}

# Criar diretório de saída se não existir
output_dir = '../outputs'
os.makedirs(output_dir, exist_ok=True)

# Salvar como Excel
with pd.ExcelWriter(f'{output_dir}/levy_analysis_{datetime.now().strftime("%Y%m%d")}.xlsx') as writer:
    for sheet_name, df in output_data.items():
        df.to_excel(writer, sheet_name=sheet_name)
        
print("\nResultados salvos com sucesso!")